# The Magic of Tree Walking

In this chapter we continue our discussion of IR design.  We introduce the idea of the Abstract Syntax Tree (AST) as an IR which is well suited for more complex languages.  We show that this intermediate representation can be directly derived from the grammar itself which facilitates the IR design. We illustrate these ideas with an interpreter that interprets our first high level language called Cuppa1.  This example also illustrates a very elegant way of processessing abstract syntax trees via something called *tree walking*.  We finish the chapter by designing a tree walker for a pretty printer with twist for our Cuppa1 language.

In [1]:
# let the notebook access the code folder
import sys
sys.path.insert(1,"code")

# Abstract Syntax Trees

Our Exp1bytecode language from the last chapter was so straightforward that the best IR was an abstract representation of the instructions which we could then directly interpret.  In more complex languages, especially higher level languages, it is usually not possible to design such simple IRs.  Instead we use Abstract Syntax Trees (ASTs).

![alt](figures/chap05/1/figure/Slide1.jpg)
<p style="text-align: center;">
Fig 1. The parse tree and corresponding abstract syntax tree for expression: `+ x - y z`.
</p>


One way to think about ASTs is as parse trees with all the derivation information deleted.  Think about the grammar,
```
exp : + exp exp
    | - exp exp
    | x
    | y
    | z
```
and consider the expression,
```
+ x - y z
```
Figure 1 shows both the parse tree and the AST for this expression.  It should be easy to see that both trees represent the same computation, namely that the difference of `y` and `z` is added to `x`.  Abstract syntax trees just represent these computations in an easier and  much more accessible way.

There is another interesting aspect of ASTs as IRs: Because every valid program has a parse tree, it is always possible to construct an AST for every valid program.  In this way ASTs are the IR of choice because it doesn’t matter how complex the programming language, there will always be an AST representation for any given valid program in that language.

## The Tuple Representation of ASTs

A convenient way to represent AST nodes is with the following structure,
```Python
(TYPE [, child1, child2,...])
```
A tree node is a tuple where the first component represents the type or name of the node followed by zero or more components each representing a child of the current node.

Consider the abstract syntax tree from Figure 1 for our expression `+ x - y x`.  That AST can be encoded in our tuple notation as follows,

In [2]:
ast = ('+', 'x', ('-', 'y', 'z'))

In order to make this easier to read the module `grammar_stuff` provides a nice function that dumps the AST in a much more readable format.

In [3]:
from grammar_stuff import dump_AST
dump_AST(ast)


(+ x 
  |(- y z))


# The Cuppa1 Programming Language

The Cuppa1 language is a high level language and borrows many of its syntactic features from the C/Java family of languages.  The language provides high-level programming language constructs such as while loops and if-then-else statements in addition to standard arighmetic infix notation for expressions.

Here is a first peek at the PLY grammar of the language.

```Python
# %load code/cuppa1_gram.py
# grammar for Cuppa1

from ply import yacc
from cuppa1_lex import tokens, lexer

# set precedence and associativity
# NOTE: all arithmetic operator need to have tokens
#       so that we can put them into the precedence table
precedence = (
              ('left', 'EQ', 'LE'),
              ('left', 'PLUS', 'MINUS'),
              ('left', 'TIMES', 'DIVIDE'),
              ('right', 'UMINUS', 'NOT')
             )


def p_grammar(_):
    '''
    program : stmt_list

    stmt_list : stmt stmt_list
              | empty

    stmt : ID '=' exp opt_semi
         | GET ID opt_semi
         | PUT exp opt_semi
         | WHILE '(' exp ')' stmt
         | IF '(' exp ')' stmt opt_else
         | '{' stmt_list '}'

    opt_else : ELSE stmt
             | empty
             
    opt_semi : ';'
             | empty

    exp : exp PLUS exp
        | exp MINUS exp
        | exp TIMES exp
        | exp DIVIDE exp
        | exp EQ exp
        | exp LE exp
        | INTEGER
        | ID
        | '(' exp ')'
        | MINUS exp %prec UMINUS
        | NOT exp
    '''
    pass

def p_empty(p):
    'empty :'
    pass

def p_error(t):
    print("Syntax error at '%s'" % t.value)

### build the parser
parser = yacc.yacc()
```

Ignore the precedence table for now.  Let's take a look at the grammar proper.  We see that programs are statement lists.  A statement list is a possibly empty list of statements and in Cuppa1 there are six different kinds of statements:

* The assigment statement assigns the value of the expression `exp` on the right side of the `=` sign to the variable `ID` on the left side of the `=` sign.

* The get statement asks the user to input an integer value which the get statement will assign to the variable `ID`. 

* The put statement prints out the value of the expression `exp` to the terminal.

* The while loop will execute `stmt` that makes up its loop body as long as the condition `exp` evaluates to true.

* The if statement will execute the statement `stmt` if the condition `exp` evaluates to true otherwise it will execute the else statement if present.

* The block statement which a list of statements appearing within curly braces.

Note that statements can be followed by an optional semicolon.

Reading further in the grammar we see that expressions are infix and Cuppa1 supports all the normal arithmetic operations.  It also supports two relational operators: equal and less equal.  Expressions can also be integer values and variable names as well as parenthesized expressions and unary minus.

It now is a good time to take look at the precedence table at the top of the grammar.  We need to tell the parser generator about the precendence and associativity of each arithmetic opertor so that the expressions are parsed correctly.  Consider the expression `3 * 4 + 1`.  Without the precedence and associativity information an LR parser would parse this as `3 * (4 + 1)` which is of course not correct knowing that multiplication has a higher precednce than addition.  If we inform the parser of the precedence and associativity of the various operators then an LR parser will parse this correctly as `(3 * 4) + 1`.  

Looking at the precedence table more closely we see that all the operators are left associative with the exception of the unary minus which is right associative.
We also see that the relational operators have the lowest precedence, addition and subtraction are at the next level, the multiplication and division operators are above that, and finally we have the unary minus at the highest level of precedence.  That is each row in the precedence table represents a precedence level and the rows are sorted in increasing precendence level.


From the lexical perspective there are no surprises.

```Python
# %load code/cuppa1_lex.py
# Lexer for Cuppa1

from ply import lex

reserved = {
    'get'   : 'GET',
    'put'   : 'PUT',
    'if'    : 'IF',
    'else'  : 'ELSE',
    'while' : 'WHILE',
    'not'   : 'NOT'
}

literals = [';','=','(',')','{','}']

tokens = [
          'PLUS','MINUS','TIMES','DIVIDE',
          'EQ','LE', 
          'INTEGER','ID',
          ] + list(reserved.values())

t_PLUS    = r'\+'
t_MINUS   = r'-'
t_TIMES   = r'\*'
t_DIVIDE  = r'/'
t_EQ      = r'=='
t_LE      = r'<='

t_ignore = ' \t'

def t_ID(t):
    r'[a-zA-Z_][a-zA-Z_0-9]*'
    t.type = reserved.get(t.value,'ID')    # Check for reserved words
    return t

def t_INTEGER(t):
    r'[0-9]+'
    return t

def t_COMMENT(t):
    r'//.*'
    pass

def t_NEWLINE(t):
    r'\n'
    pass

def t_error(t):
    print("Illegal character %s" % t.value[0])
    t.lexer.skip(1)

# build the lexer
lexer = lex.lex(debug=0)
```

Perhaps the one thing to point out is the inclusion of comments that start with `//` and then run to the end of the line are now part of the language.

Here are a couple of programs written in Cuppa1.

In [4]:
from cuppa1_gram import parser
from cuppa1_lex import lexer

In [5]:
p1 = \
'''
get x
x = x +1
put x
'''

parser.parse(p1, lexer=lexer)

In [6]:
fact = \
'''
get x;
y = 1;
while (1 <= x)
{
      y = y * x;
      x = x - 1;
}
put y;
'''

parser.parse(fact, lexer=lexer)

Here is an interesting program that loops forever but does nothing,

In [7]:
loop = "while (1) {}"

parser.parse(loop, lexer=lexer)

# The Cuppa1 Frontend

In the technical jargon of programming language implementation a *frontent* is a parser that constructs an abstract syntax tree for an input program and fills in the symbol table with basic variable information. Our Cuppa1 frontend consists of the parser specification from above enriched with embedded actions that construct tuple based AST and fills in the symbol table with variable names that defined using either the assignment statement or the get statement.

Before we get into the details of the frontend we need to introduce the Cuppa1 state.  A state in Cuppa1 is not unlike the state of the abstract machine for the Exp1bytecode.  Here the state is the computation represented by the AST together with a symbol table.  A state is an object,

In [8]:
# %load code/cuppa1_state

class State:
    def __init__(self):
        self.initialize()

    def initialize(self):
        # symbol table to hold variable-value associations
        self.symbol_table = {}

        # when done parsing this variable will hold our AST
        self.AST = None

state = State()


The member function `initialize` initializes a state object by clearing the symbol table and setting the AST to `None`.

The full frontend specification consists of the [parser](code/cuppa1_frontend_gram.py), the [lexer](code/cuppa1_lex.py), and the [state](code/cuppa1_state.py) specifications.  Here we take a look at some telling aspects of the specification.  Perhaps the best part to start with is the specification of the statements.

## Statements

In [9]:
# %load -s p_stmt code/cuppa1_frontend_gram.py
def p_stmt(p):
    '''
    stmt : ID '=' exp opt_semi
         | GET ID opt_semi
         | PUT exp opt_semi
         | WHILE '(' exp ')' stmt
         | IF '(' exp ')' stmt opt_else
         | '{' stmt_list '}'
    '''
    if p[2] == '=':
        p[0] = ('assign', p[1], p[3])
        state.symbol_table[p[1]] = None
    elif p[1] == 'get':
        p[0] = ('get', p[2])
        state.symbol_table[p[2]] = None
    elif p[1] == 'put':
        p[0] = ('put', p[2])
    elif p[1] == 'while':
        p[0] = ('while', p[3], p[5])
    elif p[1] == 'if':
        p[0] = ('if', p[3], p[5], p[6])
    elif p[1] == '{':
        p[0] = ('block', p[2])
    else:
        raise ValueError("unexpected symbol {}".format(p[1]))


Looking at the code carefully we see that the embedded actions build AST nodes for the corresponding statements.  Consider the assignment statement,
```
stmt : ID '=' exp opt_semi
```
The embedded actions for this grammar rule construct the following tuple where `p[1]` contains the name of the variable of the token `ID` and 
`p[3]` contains the AST generated by the non-terminal `exp`,
```Python
('assign', p[1], p[3])
```
In addition, the variable name in `p[1]` is written to the symbol table,
```Python
state.symbol_table[p[1]] = None
```
However, we are not doing any processing at this point so the value we record for the variable is set to `None`.  As pointed out above when we discussed the syntax of Cuppa1, the non-terminal `opt_semi` means that the statement can be followed by an optional semicolon.  

The get statement specification follows a very similar pattern.  Here is the grammar rule for get statemments,
```
stmt : GET ID opt_semi
```
The corresponding embedded actions build a tuple where `p[2]` holds the variable name,
```Python
('get', p[2])
```
Again, the variable is recorded in the symbol table with its value set to `None`,
```Python
state.symbol_table[p[2]] = None
```

The remainder of the embedded actions are pretty straight forward with the execption perhaps of the if statement.  From our discussion on the syntax of Cuppa1 above we know that if statements come in two flavors: `if-then` and `if-then-else`.  Our frontend specification has to deal with these two flavors effectively.  From the grammar snippet above we have the rule,
```
stmt : IF '(' exp ')' stmt opt_else
```
which specifies that the `if-then` statement can be followed by an optional else-part.  The tuple that is being constructed is,
```Python
('if', p[3], p[5], p[6])
```
where `p[3]` is the AST generated by the non-terminal `exp`, `p[5]` is the AST generated by the non-terminal `stmt`.
The question is, what does `p[6]` contain?  In order to answer this we need to take a look at the embedded actions for
the rule defining the non-terminal `opt_else`.

In [10]:
# %load -s p_opt_else code/cuppa1_frontend_gram.py
def p_opt_else(p):
    '''
    opt_else : ELSE stmt
             | empty
    '''
    if p[1] == 'else':
        p[0] = p[2]
    else:
        p[0] = p[1]
    

The embedded actions for the `opt_else` non-terminal specify that if an else-part exists then copy the AST of the corresponding statement.  That means that `p[6]` above will have the AST of the else-statement if the else-part exists. If the else-part does not exist then we copy whatever the non-terminal `else` produces.

For this we need to take a look at the definition of the non-terminal `empty`.

In [11]:
# %load -s p_empty code/cuppa1_frontend_gram.py
def p_empty(p):
    'empty :'
    p[0] = ('nil',)


The embedded action for `empty` constructs a tuple with no children and of the type `nil`.  That means, `p[6]` in the embedded action for the if statement will have the node,
```Python
('nil',)
```
indicating that there is no else-part.  This is worthwhile to remember because later on when we actually walk the tree in order to do some processing we need to be aware of these to distinct tree patterns.

## Statement Lists and Programs

Statement lists and programs are specified as follows in the frontend,

In [12]:
# %load -s p_prog,p_stmt_list code/cuppa1_frontend_gram.py
def p_prog(p):
    '''
    program : stmt_list
    '''
    state.AST = p[1]

#########################################################################
def p_stmt_list(p):
    '''
    stmt_list : stmt stmt_list
              | empty
    '''
    if (len(p) == 3):
        p[0] = ('seq', p[1], p[2])
    elif (len(p) == 2):
        p[0] = p[1]



Programs in Cuppa1 are lists of statements.  The embedded action for  the rule `program : stmt_list`  will copy
the AST computed by the non-terminal `stmt_list` in `p[1]` into the AST field of the state.

The rule for statement lists,
```
stmt_list : stmt stmt_list
          | empty

```
states that statement lists consist of a stament followed by statement lists or they are empty.  
For non-empty statement lists we need a way to glue all the different statements together in the AST.
The traditional way to do this is with a 'sequennce' node.
Let's take a look at the embedded action for non-empty statement lists,
```Python
p[0] = ('seq', p[1], p[2])
```
Here we construct an AST node of type `seq` where the first child is the AST computed by the non-terminal `stmt` and
the second child is the AST of the rest of the statement list computed by the non-terminal `stmt_list`.  

If the statement list is empty then the corresponding embedded action,
```Python
p[0] = p[1]
```
will copy the AST computed by the non-terminal `empty`.  We know from our previous disscussion that this AST is the tree node `('nil',)`.

Now, for a program with two statements,
```
Stmt1
Stmt2
```
our frontend will compute the following AST,
```Python
('seq',
    <Stmt1>,
    ('seq',
       <Stmt2>,
       ('nil',)))
```
where `<Stmt1>` and `<Stmt2>` represent the ASTs for the corresponding program statements `Stmt1` and `Stmt2`.
It is easy to see that statement lists are `nil` terminated lists constructed with `seq` nodes.  We will study this in more detail once we finished our discussion of the frontend design.


## Expressions

We start our discussion of expressions in Cuppa1 with the binary operators,

In [13]:
# %load -s p_binop_exp code/cuppa1_frontend_gram.py
def p_binop_exp(p):
    '''
    exp : exp PLUS exp
        | exp MINUS exp
        | exp TIMES exp
        | exp DIVIDE exp
        | exp EQ exp
        | exp LE exp
    '''
    p[0] = (p[2], p[1], p[3])
 


For each binary operator the embedded action constructs an AST node of type `binop`,
```
p[0] = (p[2], p[1], p[3])
```
where `p[2]` represents the operator name and `p[1]` and `p[2]` the AST of left and right subexpressions, respectively.  This is identical to the scheme of representing expressions developed for the Exp1bytecode interpreter in Chapter 4.  

The remaining expression should look familiar at this point.  They are almost identical to the scheme we developed
for the Exp1bytecode interpreter with the exception of the occasional name change,

In [14]:
# %load -s p_integer_exp,p_id_exp,p_paren_exp,p_uminus_exp,p_not_exp code/cuppa1_frontend_gram.py
def p_integer_exp(p):
    '''
    exp : INTEGER
    '''
    p[0] = ('integer', int(p[1]))
    
#########################################################################

def p_id_exp(p):
    '''
    exp : ID
    '''
    p[0] = ('id', p[1])

#########################################################################

def p_paren_exp(p):
    '''
    exp : '(' exp ')'
    '''
    p[0] = ('paren', p[2])
    
#########################################################################

def p_uminus_exp(p):
    '''
    exp : MINUS exp %prec UMINUS
    '''
    p[0] = ('uminus', p[2])

#########################################################################

def p_not_exp(p):
    '''
    exp : NOT exp
    '''
    p[0] = ('not', p[2])

#########################################################################


## Generating ASTs

We are now in a position to take a look at the AST our Cuppa1 frontend will generate for an input program.

First, let's load the necessary modules.

In [15]:
from cuppa1_frontend_gram import parser
from cuppa1_lex import lexer
from cuppa1_state import state
from grammar_stuff import dump_AST

Let's start with something simple,

In [16]:
state.initialize()
parser.parse("get x; put x", lexer=lexer)

In [17]:
dump_AST(state.AST)


(seq 
  |(get x) 
  |(seq 
  |  |(put 
  |  |  |(id x)) 
  |  |(nil)))


In [18]:
state.symbol_table

{'x': None}

As expected, the AST generated for our program is the `nil` terminated `sequence` of the get and put statements. The symbol table has an entry for `x` since the get statement defines this variable by assigning it the value the user would input if the program were executed.

Let's put an assignment statement between the get and the put,

In [19]:
state.initialize()
parser.parse("get x; x = x + 1; put x", lexer=lexer)

In [20]:
dump_AST(state.AST)


(seq 
  |(get x) 
  |(seq 
  |  |(assign x 
  |  |  |(+ 
  |  |  |  |(id x) 
  |  |  |  |(integer 1))) 
  |  |(seq 
  |  |  |(put 
  |  |  |  |(id x)) 
  |  |  |(nil))))


In [21]:
state.symbol_table

{'x': None}

As one would expect, the sequence of statements now also contains the assignment statement.  The symbol table still only as the variable `x` in it because it is the only variable being defined by this program.

Let's try our degenerate while loop program from above -- the one that loops forever and does nothing,

In [22]:
state.initialize()
parser.parse("while (1) {}", lexer=lexer)

In [23]:
dump_AST(state.AST)


(seq 
  |(while 
  |  |(integer 1) 
  |  |(block 
  |  |  |(nil))) 
  |(nil))


It is perhaps intesting to look at the AST for the while loop.  The first child of the while node is the condition expression of the loop here set to the integer 1.  The second child is the body of the loop represented by a block statement with no statements.

The next two programs deal with if statements.  The first program is a simple if statement without an else part.
Notice that the missing else part is represented by a `nil` node.  The second program has an if-then-else statement. Here all three children are present.  

In [24]:
state.initialize()
parser.parse("get x; if (0 <= x) put 1", lexer=lexer)

In [25]:
dump_AST(state.AST)


(seq 
  |(get x) 
  |(seq 
  |  |(if 
  |  |  |(<= 
  |  |  |  |(integer 0) 
  |  |  |  |(id x)) 
  |  |  |(put 
  |  |  |  |(integer 1)) 
  |  |  |(nil)) 
  |  |(nil)))


In [26]:
parser.parse("get x; if (0 <= x) put 1 else put 2", lexer=lexer)

In [27]:
dump_AST(state.AST)


(seq 
  |(get x) 
  |(seq 
  |  |(if 
  |  |  |(<= 
  |  |  |  |(integer 0) 
  |  |  |  |(id x)) 
  |  |  |(put 
  |  |  |  |(integer 1)) 
  |  |  |(put 
  |  |  |  |(integer 2))) 
  |  |(nil)))


Finally, let's try something more complicated.  Here we look at a program from the Cuppa1 examples that computes the factorial value of `x` in the variable `y`.

In [28]:
from cuppa1_examples import fact

In [29]:
print(fact)


get x;
y = 1;
while (1 <= x)
{
      y = y * x;
      x = x - 1;
}
put y;



In [30]:
state.initialize()
parser.parse(fact, lexer=lexer)

In [31]:
dump_AST(state.AST)


(seq 
  |(get x) 
  |(seq 
  |  |(assign y 
  |  |  |(integer 1)) 
  |  |(seq 
  |  |  |(while 
  |  |  |  |(<= 
  |  |  |  |  |(integer 1) 
  |  |  |  |  |(id x)) 
  |  |  |  |(block 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(assign y 
  |  |  |  |  |  |  |(* 
  |  |  |  |  |  |  |  |(id y) 
  |  |  |  |  |  |  |  |(id x))) 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(assign x 
  |  |  |  |  |  |  |  |(- 
  |  |  |  |  |  |  |  |  |(id x) 
  |  |  |  |  |  |  |  |  |(integer 1))) 
  |  |  |  |  |  |  |(nil))))) 
  |  |  |(seq 
  |  |  |  |(put 
  |  |  |  |  |(id y)) 
  |  |  |  |(nil)))))


In [32]:
state.symbol_table

{'x': None, 'y': None}

The AST is starting to get pretty big.  You should try to walk through the AST and trace every component of the AST back to the language construct in the `fact` source program.  The symbol table has the variables `x` and `y` in it which is not surprising because those are the two variables being defined by get/assignment statements.

# Tree Walking

Trees are recursive data structures where each node in the tree other than leaf nodes is defined in terms of other tree nodes.  Consider a binary tree for example.  Here each node in the tree is defined in terms of a left and a right tree node and these nodes in turn are binary tree nodes themselves defined in terms of left and right nodes and so on until we reach the leaf nodes.

This recursive structure of trees gives rise to an elegant way of processing trees: *tree walking*.  A tree walker typically starts at the root node and traverses the tree in a depth first manner (Figure 2).

![alt](figures/chap05/2/figure/Slide1.jpg)
<p style="text-align: center;">
Fig 2. An abstract syntax tree for the expression `3*2+4`.  The red arrows indicate a typical tree walker traversal.
</p>


To make this processing computationally feasible in Python we map ASTs to our tuple notation.  For the tree in Figure 2 the tuple notation is,

In [33]:
ast = ('+', ('*', ('integer', 3), ('integer', 2)), ('integer', 4))

In order to make it look pretty we can use our `dump_AST` function,

In [34]:
from grammar_stuff import dump_AST

dump_AST(ast)


(+ 
  |(* 
  |  |(integer 3) 
  |  |(integer 2)) 
  |(integer 4))


## A Simple Tree Walker

A clever way to implement a tree walker is to essentially mirror the recursive nature of the tree in the walker itself.  That is, for every type of tree node the walker provides a processing routine that knows how to deal with the structure of that node, it doesn't matter if it is an internal tree node or a leaf node.  If it is an internal node then the walker calls itself recursively on the child nodes.  Consider that we want to evaluate the AST for the expression above to its integer value.  Our expression tree has three types of nodes: addition, multiplication, and integer constant nodes.  We can define a tree walker to do this as follows,

In [35]:
def const(node):
    # pattern match the constant node
    (INTEGER, val) = node
    
    # return the value as an integer value
    return int(val)

def add(node):
    # pattern match the tree node
    (ADD, left, right) = node
    
    # recursively call the walker on the children
    left_val = walk(left)
    right_val = walk(right)
    
    # return the sum of the values of the children
    return left_val + right_val

def mult(node):
    # pattern match the tree node
    (MULT, left, right) = node
    
    # recursively call the walker on the children
    left_val = walk(left)
    right_val = walk(right)
    
    # return the product of the values of the children
    return left_val * right_val


Notice that our walker has one function for each node type in our AST.  Also notice that we are using Python's ability to pattern match tuples in order to extract the components of the various tree node types.
The question now is, how do we get the tree walker to call the appropriate function for the node that it is currently processing?  The answer is a dispatch dictionary for AST node types.

In [36]:
dispatch_dict = {
    '+' : add,
    '*' : mult,
    'integer' : const
}

The keys in the dispatch dictionary are the node types of our AST and the values are the functions that can process each of those tree node types respectively.  That is, the dispatch dictionary associates node types with their corresponding functions.

Finally, we are in a position to define our tree walker proper.

In [37]:
def walk(node):
    # first component of any tree node is its type
    t = node[0]
    
    # lookup the function for this node
    node_function = dispatch_dict[t]
    
    # now call this function on our node and capture the return value
    val = node_function(node)
    
    return val

If you look carefully at this tree walker you will notice that it implements precisely the depth-first traversal given in Figure 2 by recursively calling the `walk` function which in turn will dispatch the appropriate node function.

Let's walk our AST for expression `3*2+4`,

In [38]:
print(walk(ast))

10


The walker returns the value `10` as we would expect from the evaluation of this term using the standard rules of arithmetic.

<!-- TODO: movie for this interpretation tree walkder -->

## Tree Walkers are Plug'n Play

Tree walkers exist completely separately from the AST.  Sure, they have to follow the rules of the recursively nature of the tree they are supposed to evaluate but other than that walkers are completely separate from the definition of an AST.  From an architectural perspective we can view this as depicted in Figure 3.  The walker plugs into the AST and processes it using its node functions.

![alt](figures/chap05/3/figure/Slide1.jpg)
<p style="text-align: center;">
Fig 3. The high-level architecture of a tree walker.
</p>


This gives rise to the idea of 'plug'n play' for tree walkers: In order to process an AST we select/build an appropriate tree walker and plug it into the AST and start processing.

Taking this idea a step further, there is nothing to prevent us from plugging in multiple walkers during the processing of an AST, each performing a distinct phase of the processing.  Figure 4 illustrates this approach.  Here the frontend builds the AST, the walkers process the AST and the final walker generates the appropriate output. At the end of this chapter we will encounter a pretty printer for Cuppa1 that does exactly this: it has multiple walkers each performing a distinct task and the last one generates the pretty printed output.

![alt](figures/chap05/4/figure/Slide1.jpg)
<p style="text-align: center;">
Fig 4. Tree walker plug'n play, processing an AST with multiple walkers.
</p>


# An Interpreter for Cuppa1

We now have all the machinery in place to write a tree walking based interpreter for Cuppa1.  The tree walker follows the structure of the walker outlined in the previous section: it consists of a walk function, a dispatch dictionary, and a set of node functions (one for each AST node type).  Conceptually we can think of the architecture as shown in Figure 5.  The frontend builds the AST and our tree walker interprets the AST and produces the output of the program execution.

![alt](figures/chap05/5/figure/Slide1.jpg)
<p style="text-align: center;">
Fig 5. The architecture of the Cuppa1 interpreter.
</p>


Let's start by looking at the `walk` function and the dispatch dictionary of the tree walker.

In [39]:
from cuppa1_interp_walk import *
from grammar_stuff import assert_match

In [40]:
# %load -r 212-247 code/cuppa1_interp_walk.py
#########################################################################
# walk
#########################################################################
def walk(node):
    # node format: (TYPE, [child1[, child2[, ...]]])
    type = node[0]
    
    if type in dispatch_dict:
        node_function = dispatch_dict[type]
        return node_function(node)
    else:
        raise ValueError("walk: unknown tree node type: " + type)

# a dictionary to associate tree nodes with node functions
dispatch_dict = {
    'seq'     : seq,
    'nil'     : nil,
    'assign'  : assign_stmt,
    'get'     : get_stmt,
    'put'     : put_stmt,
    'while'   : while_stmt,
    'if'      : if_stmt,
    'block'   : block_stmt,
    'integer' : integer_exp,
    'id'      : id_exp,
    'paren'   : paren_exp,
    '+'       : plus_exp,
    '-'       : minus_exp,
    '*'       : times_exp,
    '/'       : divide_exp,
    '=='      : eq_exp,
    '<='      : le_exp,
    'uminus'  : uminus_exp,
    'not'     : not_exp
}


The `walk` function accesses the node type information and uses this to look up the appropriate node function in the dispatch dictionary.  The node function is then applied to the node itself.  With the exception of additional error handling this works exactly the same way as the expression walker defined in the previous section.

A good function to start our discussion of node functions is the node function for assignment statements.

In [41]:
# %load -s assign_stmt code/cuppa1_interp_walk.py
def assign_stmt(node):

    (ASSIGN, name, exp) = node
    assert_match(ASSIGN, 'assign')
    
    value = walk(exp)
    state.symbol_table[name] = value


The function pattern matches the assignment tree node.  Here we have some additional error checking in the form of the `assert_match` function that makes sure the the type of the current tree node is really the type that we are expecting (By the way, defensive programming of this kind is a good way to help you debug your tree walkers).  The match extracts the name of the variable of the assignment and the AST for the expression.
The expression needs to be evaluated in order to compute a value to be assigned to the variable.  To compute the value of the the expression we walk it!  Once we have a value then we can associate the variable with the value in the symbol table. And this is all there is to it! Because we have defined a node function for each type of node that can appear in the AST it seems almost like magic that all you have to do is call the `walk` function to get a value back from an expression.  Note that in Cuppa1 statements do not have any return value, only expressions have return values.

What does the interpretation of `seq` nodes look like?  Let's take a look,

In [42]:
# %load -s seq code/cuppa1_interp_walk.py
def seq(node):
    
    (SEQ, stmt, stmt_list) = node
    assert_match(SEQ, 'seq')
    
    walk(stmt)
    walk(stmt_list)


The first child of a `seq` node is the AST of a statement and the second child is the rest of the sequence of statements.  In order to interpret a `seq` node we walk the statement and then we walk the rest of the list.

The interpretation of the `nil` node is probably another interesting case to look at,

In [43]:
# %load -s nil code/cuppa1_interp_walk.py
def nil(node):
    
    (NIL,) = node
    assert_match(NIL, 'nil')
    
    # do nothing!
    pass


The intpretation of a `nil` node is to do nothing!

Let's take a look at the interpretation of a while loop.

In [44]:
# %load -s while_stmt code/cuppa1_interp_walk.py
def while_stmt(node):

    (WHILE, cond, body) = node
    assert_match(WHILE, 'while')
    
    value = walk(cond)
    while value != 0:
        walk(body)
        value = walk(cond)


The interpretation is probably just the way you had imagined.  We walk the condition of the loop before the try to execute in order to obtain a value for the condition.  Then, if the value of the condition is not equal to zero, *i.e.* false in our interpretation, we execute the loop body by -- you guessed it -- walking it.  Once we executed the body we walk the condition again to see if we should continue to execute the loop.  Once the condition evaluates to something non-zero the loop stops and we return from this node function.  Again, as we mentioned above statements in Cuppa1 do not have any return values.

Perhaps the most complicated statement to interpret is the if statement because it comes in two flavors: one with the else part and one without the else part.  Our node function has to recognize which flavor it is dealing with and then perform the right interpretation.

In [45]:
# %load -s if_stmt code/cuppa1_interp_walk.py
def if_stmt(node):
    
    try: # try the if-then pattern
        (IF, cond, then_stmt, (NIL,)) = node
        assert_match(IF, 'if')
        assert_match(NIL, 'nil')

    except ValueError: # if-then pattern didn't match
        (IF, cond, then_stmt, else_stmt) = node
        assert_match(IF, 'if')

        value = walk(cond)
        if value != 0:
            walk(then_stmt)
        else:
            walk(else_stmt)
        return
 
    else: # if-then pattern matched
        value = walk(cond)
        if value != 0:
            walk(then_stmt)
        return


This node function tries to pattern match the if statement flavor without the else part with a try-except-else statement.  Notice the pattern with the `nil` node.  If the pattern match is successful then Python will execute the else part of the try statement.  Here, if the condition of the if statement is non-zero the node function will execute the then part of the if statement otherwise it will just return.

Now, if the if-then pattern match was not successful that means the else part of the if statement is not a `nil` node.
In this case we pattern match the if-then-else tree node, evaluate the condition, and then execute the appropriate statement: the then part if the condition is non-zero, the else part if the condition was zero.

The last two things we should look at is the interpretation of an arithmetic and a relational operator.  Consider the `<=` operator,

In [46]:
# %load -s le_exp code/cuppa1_interp_walk.py
def le_exp(node):
    
    (LE,c1,c2) = node
    assert_match(LE, '<=')
    
    v1 = walk(c1)
    v2 = walk(c2)
    
    return 1 if v1 <= v2 else 0


After pattern matching this node function walks the left and right children of our operator tree node to obtain the values for the children expressions.  Lucky for us Python directly implements the `<=` operator so we can use it for our interpretation but we need to take care of the return values.  Cuppa1 does not deal with `True/False` Boolean values but with non-zero for a true value and zero for a false value.  So we need to map the Boolean values the the Python `<=` operator returns into the proper return values for Cuppa1.

The interpretation of the addition operator works in a very similar fashion except that we add the values rather than performing a relational operation on them,

In [47]:
# %load -s plus_exp code/cuppa1_interp_walk.py
def plus_exp(node):
    
    (PLUS,c1,c2) = node
    assert_match(PLUS, '+')
    
    v1 = walk(c1)
    v2 = walk(c2)
    
    return v1 + v2


The full [implementation](code/cuppa1_interp_walk.py) of the tree walker.

## Running the Interpreter

In order to run the interpreter we'll provide a interpretation function that pulls building the AST, dealing with the state, and running the walking together.

```python
# %load code/cuppa1_interp.py
#!/usr/bin/env python
# Cuppa1 interpreter

from argparse import ArgumentParser
from cuppa1_lex import lexer
from cuppa1_frontend_gram import parser
from cuppa1_state import state
from cuppa1_interp_walk import walk

def interp(input_stream):

    # initialize the state object
    state.initialize()

    # build the AST
    parser.parse(input_stream, lexer=lexer)

    # walk the AST
    walk(state.AST)

if __name__ == "__main__":
    # parse command line args
    aparser = ArgumentParser()
    aparser.add_argument('input')

    args = vars(aparser.parse_args())

    f = open(args['input'], 'r')
    input_stream = f.read()
    f.close()

    # execute interpreter
    interp(input_stream=input_stream)
```

Once `interp` has an input stream it initializes the state object, builds the AST for the input stream, and then walks the AST.  Let's try this on some programs.

In [77]:
from cuppa1_interp import interp

In [49]:
interp("get x; x = x + 1; put x")

Value for x? 3
> 4


In [50]:
from cuppa1_examples import *

In [51]:
print(list)


// list of integers
get x
while (1 <= x) 
{
    put x;
    x = x + - 1;
    i = x
}



In [52]:
interp(list)

Value for x? 5
> 5
> 4
> 3
> 2
> 1


In [53]:
print(fact)


get x;
y = 1;
while (1 <= x)
{
      y = y * x;
      x = x - 1;
}
put y;



In [54]:
interp(fact)

Value for x? 3
> 6


In [55]:
print(ifex)


// logical not in Cuppa1
get x
if (x == 0)
   put 1
else 
  put 0



In [56]:
interp(ifex)

Value for x? 1
> 0


In [57]:
print(logical_and)


// logical and in Cuppa1 can be simulated with multiplication
x = 0
y = 0
put x
put y
put x * y

x = 1
y = 0
put x
put y
put x * y

x = 0
y = 1
put x
put y
put x * y

x = 1
y = 1
put x
put y
put x * y



In [58]:
interp(logical_and)

> 0
> 0
> 0
> 1
> 0
> 0
> 0
> 1
> 0
> 1
> 1
> 1


In [59]:
print(logical_or)


// logical or in Cuppa1 can be simulated with addition
x = 0
y = 0
put x
put y
put x + y

x = 1
y = 0
put x
put y
put x + y

x = 0
y = 1
put x
put y
put x + y

x = 1
y = 1
put x
put y
put not not (x + y) // make it look like '1'



In [60]:
interp(logical_or)

> 0
> 0
> 0
> 1
> 0
> 1
> 0
> 1
> 1
> 1
> 1
> 1


# A Cuppa1 Pretty Printer with a Twist

We can apply tree walking techniques to a Cuppa1 pretty printer.  But this time it's a pretty printer with a twist, 

> The pretty printer will output a pretty printed version of a Cuppa1 input program and it will flag assignment/get statements to variables which are not used in the program!

For example, the Cuppa1 program that prints out a list of integers,
```
get x; while (1 <= x) { put x; x = x + - 1; i = x }
```
will be pretty printed as,
```
get x
while (1 <= x)
{
   put x
   x = x + -1
   i = x // *** i is not used ***
}
```
with `i` flagged as not being used in the program.  Just notice that this is not possible to do in a syntax-directed manner: usage points are almost always non-local to definition points

## Architecture of the Cuppa1 Pretty Printer

The architecture of our pretty printer consists of our Cuppa1 frontend and two tree walkers.  One tree walker to look for variable usage points in expressions that marks the symbol table appropriately.  The other tree walker uses the AST to generate pretty printed code.  Figure 6 illustrates that architecture.  Our Cuppa1 frontend buils the AST and the two tree walkers process the AST.

![alt](figures/chap05/6/figure/Slide1.jpg)
<p style="text-align: center;">
Fig 6. The architecture of the Cuppa1 pretty printer with a twist.
</p>


### The First Pass Walker

The first pass of the pretty printer walks the AST and looks for variables in expressions because only those count as usage points.  A peek at the [tree walker for the first pass](code/cuppa1_pp1_walk.py)  shows that it literally just walks the tree doing nothing until it finds a variable in an expression.  If it finds a variable in an expression then the node function for `ID` marks the variable in the symbol table as used,

In [61]:
# %load -s id_exp code/cuppa1_pp1_walk.py
def id_exp(node):
    
    (ID, name) = node
    assert_match(ID, 'id')
    
    # we found a use scenario of a variable, if the variable is defined
    # set it to true
    if name in state.symbol_table:
        state.symbol_table[name] = True


Recall that when the frontend finds a definition of a variable, that is, either an assignment or a get statement for a particular variable it enters it into the symbol table and initializes it with `None`,

```Python
# %load -s p_stmt code/cuppa1_frontend_gram.py
def p_stmt(p):
    '''
    stmt : ID '=' exp opt_semi
         | GET ID opt_semi
         | PUT exp opt_semi
         | WHILE '(' exp ')' stmt
         | IF '(' exp ')' stmt opt_else
         | '{' stmt_list '}'
    '''
    if p[2] == '=':
        p[0] = ('assign', p[1], p[3])
        state.symbol_table[p[1]] = None
    elif p[1] == 'get':
        p[0] = ('get', p[2])
        state.symbol_table[p[2]] = None
    ...
```

We can test our walker,

In [62]:
from cuppa1_frontend_gram import parser
from cuppa1_lex import lexer
from cuppa1_pp1_walk import walk as pp1_walk
from cuppa1_state import state

In [63]:
state.initialize()

In [64]:
parser.parse("get x", lexer=lexer)

In [65]:
pp1_walk(state.AST)

In [66]:
state.symbol_table

{'x': None}

So, our small program defines `x` but never uses it according to our symbol table.  Let's try this again with a slightly more complicated program,

In [67]:
state.initialize()

In [68]:
parser.parse("get x; put x", lexer=lexer)

In [69]:
pp1_walk(state.AST)

In [70]:
state.symbol_table

{'x': True}

Good, our symbol table shows the fact that `x` was defined and used in the program.

### The Second Pass Walker

The tree walker for the second pass walks the AST and compiles a formatted string that represents the pretty printed program.  This is not unlike the pretty printer for Exp1 which we looked at in Chapter 3.  The difference of course is that here we are not doing this in a syntax-directed manner but by tree walking but we are still generating a formatted string.

For example, walking the `seq` node generates a string put together by the string generated for the statement in the `seq` node and the the string from the rest of the program,

In [71]:
# %load -s seq code/cuppa1_pp2_walk.py
def seq(node):

    (SEQ, s1, s2) = node
    assert_match(SEQ, 'seq')
    
    stmt = walk(s1)
    list = walk(s2)

    return stmt + list


A look at the node function for the `while` node is also interesting,

In [72]:
# %load -s while_stmt code/cuppa1_pp2_walk.py
def while_stmt(node):
    global indent_level
    
    (WHILE, cond, body) = node
    assert_match(WHILE, 'while')
    
    cond_code = walk(cond)

    indent_level += 1
    body_code = walk(body)
    indent_level -= 1

    code = indent() + 'while (' + cond_code + ')\n' + body_code

    return code


Here we generate code for the condition and the body and then generate the code for the whole `while`.  Notice that we keep careful track of the indentation level so that the output code looks nice.  Now, let's see what happens at the variable definitions, that is the assignment and get statement nodes,

In [73]:
# %load -s assign_stmt,get_stmt code/cuppa1_pp2_walk.py
def assign_stmt(node):

    (ASSIGN, name, exp) = node
    assert_match(ASSIGN, 'assign')
    
    exp_code = walk(exp)

    code = indent() + name + ' = ' + exp_code
    
    if not state.symbol_table[name]:
        code += ' // *** '+ name + ' is not used ***'

    code += '\n'
    return code

#########################################################################

def get_stmt(node):

    (GET, name) = node
    assert_match(GET, 'get')

    code = indent() + 'get ' + name
    
    if not state.symbol_table[name]:
        code += ' // '+ name + ' is not used'

    code += '\n'
    return code


Here we generate code for both types of nodes as you would expect.  However, the twist is that we insert a comment about the variable not being used if the symbol table entry for the variable being defined by either an assignment or get statement is still set to `None`.  A value of `None` for the variable being defined indicates that the walker for the first pass did not find a usage point for this variable.

The full implementation of [phase 1](code/cuppa1_pp1_walk.py) and [phase 2](code/cuppa1_pp2_walk.py) of the tree walker.

## Testing the Pretty Printer

To make testing easier we provide a function that ties the frontend, the first pass, and the second pass together.

In [74]:
# %load code/cuppa1_pp.py
#!/usr/bin/env python
# Cuppa1 pretty printer

from sys import stdin
from cuppa1_frontend_gram import parser
from cuppa1_lex import lexer
from cuppa1_state import state
from cuppa1_pp1_walk import walk as pp1_walk
from cuppa1_pp2_walk import walk as pp2_walk
from cuppa1_pp2_walk import init_indent_level

def pp(input_stream = None):

    # if no input stream was given read from stdin
    if not input_stream:
        input_stream = stdin.read()

    # initialize the state object and indent level
    state.initialize()
    init_indent_level()

    # build the AST
    parser.parse(input_stream, lexer=lexer)

    # walk the AST
    pp1_walk(state.AST)
    code = pp2_walk(state.AST)

    # output the pretty printed code
    print(code)

if __name__ == "__main__":
    # execute only if run as a script
    pp()


Let's try it on the example that we started this section with,

In [79]:
from cuppa1_pp import pp

In [80]:
pp("get x; while (1 <= x) { put x; x = x + - 1; i = x }")

get x
while (1 <= x)
{
   put x
   x = x + -1
   i = x // *** i is not used ***
}



So it works as expected.  However, our simple scheme of finding variable definition points messes up in some cases. Consider,

In [76]:
funny = \
'''
y = x
get x
put y
'''

pp(funny)

y = x
get x
put y



Here the `get x` statement should have been flagged because that value of `x` is never used.  But because we are not careful in remembering the order in which definitions and usage of variables occur in our pretty printer it doesn't flag it. 

# Summary

We started the chapter with a discussion of abstract syntax trees (ASTs) as an IR which is well suited for more complex languages. We show that this intermediate representation can be directly derived from the grammar or more precisely from the parse trees a grammar generates.  We used these ideas to construct an interpreter for our first high-level language Cuppa1.  We used tree walking as a technique to interpret the AST generated by the frontend.  The frontend is the part of a language processor that generates the AST and usually fills out the symbol table with some basic information about the symbols/variables it encounters while processing the input program.  We looked at the architecture of tree walkers in some detail.  These usually reflect the recursive nature of the ASTs.  The basic components of a tree walker is a dispatcher and a set of node functions; one for each type of node that can appear in the AST.   We finished the chapter by designing a tree walker for a pretty printer with twist for our Cuppa1 language.  This pretty printer flagged unused but defined variables in the generated output program.

# Notes

A nice overview of abstract syntax tree design appears in (Jones, 2003).  In (Wang *et al*, 1997) the abstract syntax is described in terms of a abstract syntax description language which is well suited for large scale language implementation projects.  A paper that discusses the relationship between abstract and concrete syntax is (Wile, 1997).  Wikipedia has nice articles on [abstract syntax trees](https://en.wikipedia.org/wiki/Abstract_syntax_tree) and
[tree walking (traversal)](https://en.wikipedia.org/wiki/Tree_traversal).


Jones, J. (2003, September). [*Abstract syntax tree implementation idioms*](http://www.hillside.net/plop/plop2003/Papers/Jones-ImplementingASTs.pdf). In Proceedings of the 10th conference on pattern languages of programs (plop2003) (p. 26).

Wang, D. C., Appel, A. W., Korn, J. L., & Serra, C. S. (1997, October). [*The Zephyr Abstract Syntax Description Language*](https://www.usenix.org/legacy/publications/library/proceedings/dsl97/full_papers/wang/wang.pdf). In DSL (Vol. 97, pp. 17-17).

Wile, D. S. (1997, May). [*Abstract syntax from concrete syntax*](https://pdfs.semanticscholar.org/7bdf/f33b709824c0745d995fc922840e05cd151d.pdf). In Proceedings of the 19th international conference on Software engineering (pp. 472-480). ACM.

# Exercises

1. Extend the Cuppa1 frontend with the proper logical operators `and` and `or`, then
  a. implement them in the Cuppa1 interpreter
  b. implement them in the Cuppa1 pretty printer
 
1. Extend the Cuppa1 frontend to allow for user defined strings in `put` and `get`.  For example, 
   ```
   get "Please enter the radius: " r
   ```
   and 
   ```
   put "The value of x is: " x
   ```
   Implement this in the Cuppa1 interpreter.

1. Extend the Cuppa1 pretty printer so that it prints out a message for variables that are being used before being defined.  Here *defined* means a variable that has been assigned a value through either an assignment or get statement.  For example, in the program,
   ```
   i = x
   put i
   ```
   the pretty printer should flag the variable `x` as being used before being defined.

1. Refine the Cuppa1 pretty printer so that variables that are in programs such as,
   ```
   y = x
   get x
   put y
   ```
   are properly flagged.  Here the variable `x` on the second line should be flagged as not used because it is being set but then not used in the remainder of the program.
   
1. Given the [lexer](code/ubasic_lex.py) and the [parser specification](code/ubasic_gram.py) for uBasic (micro-Basic) in the [code](code) folder develop an interpreter for this language.



